In [56]:
#Building a Vertical Search Engine for 7071CEM - Information Retrieval Coursework
#Importing the relevant crawling libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

In [ ]:
#Building the Crawler
pubdata = []
Ptitle = []

for i in range(0, 5):
  url = f"https://pureportal.coventry.ac.uk/en/organisations/research-centre-for-computational-science-and-mathematical-modell/publications/?page={i}"
  page = requests.get(url)
  time.sleep(5)
  soup = BeautifulSoup(page.content, "html.parser")
  whole = soup.find_all("div", class_="result-container")

  for info in whole:

    title = info.find("h3", class_="title")
    for link in title:
      pubtitle = link.text
      Ptitle.append(pubtitle)
      publink = link["href"]

    authors = info.find_all("a", class_ = "link person")
    for authinfo in authors:
      authname = authinfo.text
      authlink = authinfo["href"]
      
    date = info.find("span", class_="date").text
   
    pubdata.append([[pubtitle], [publink], [authname], [authlink], [date]])
    
print(len(pubdata))
print(pubdata)
#type(pubtitle) #to output the variable type which is str (string)

In [ ]:
#Compiling the data into a dataframe to display and export to csv

table = pd.DataFrame(pubdata, columns=["Title", "Publication Link", "Authors", "Pureportal Link", "Publication Date"])
table

#table.to_csv("InfoRetrievalCW.csv") #to export to csv
#The csv file has been uploaded to a OneDrive folder and the link will be provided in the report

In [ ]:
#List of all Publication Titles as crawled from site.
print(len(Ptitle))

#Datatype of Ptitle
type(Ptitle)

In [ ]:
#Importing Libraries for preprocessing and cleaning
import nltk
import string
nltk.download('punkt')
nltk.download('wordnet')
nltk.download("stopwords")
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
SW = stopwords.words("english")
Lstem = WordNetLemmatizer()

In [ ]:
# A document from the corpus that contains punctuation 
print(Ptitle[8])

In [ ]:
#Removing Punctuation
Pub_Title = ["".join(punc for punc in sent
                     if punc not in string.punctuation)
            for sent in Ptitle]

print(Pub_Title[8])
len(Pub_Title)

In [ ]:
#Preprocessing the data and building the inverted index
invert_indx = defaultdict(set)

for docxid, c in enumerate(Pub_Title):
  for word in word_tokenize(c):
    word_lower = word.lower()
    if word_lower not in SW:
      word_stem = Lstem.lemmatize(word_lower)
      invert_indx[word_stem].add(docxid)

print(len(invert_indx)) #outputs the total number of keywords
print(invert_indx) #displays the inverted index for every word in our corpus

In [64]:
#This section of code does 3 things:
#Defines a function called query_processor that accepts a value "query"
#Preprocesses the query to match the inverted index format
#Returns the index values if the "query" matches any word in our index set() and an empty set() if it does not

def query_processor(query):
  matching_docx = set()
  for word in word_tokenize(query):
    word_lower = word.lower()
    if word_lower not in SW and string.punctuation:
      word_Lstem = Lstem.lemmatize(word_lower)
      matches = invert_indx.get(word_Lstem)
      if matches:
       matching_docx |= matches
      else:
        print("Sorry, we got nothing.")

  return matching_docx

In [ ]:
#Testing our search engine with inputs or "queries"
#Testing with stopwords
query_processor("ecg")

In [ ]:
print(Ptitle[34]) #to display the titles corresponding to the output gotten from the processor
#print(Ptitle[88])
#print(Ptitle[187])